In [1]:
import numpy as np
import random

class OnlineMatchingModel:
    def __init__(self, N, T, E, rewards, usage_duration_distributions):
        # Initialize the model parameters
        self.N = N  # Number of offline vertices (resources)
        self.T = T  # Number of online vertices (requests)
        self.E = E  # Edges in the bipartite graph
        self.rewards = rewards  # Reward for each resource
        self.usage_duration_distributions = usage_duration_distributions  # Usage duration distributions

    def greedy_algorithm(self):
        total_reward = 0
        resource_availability = [0] * self.N  # Track when resources become available
        for t in range(self.T):
            available_resources = [i for i in range(self.N) if t >= resource_availability[i]]
            # Find the highest reward resource that can be matched
            best_resource = None
            best_reward = 0
            for i in available_resources:
                if (i, t) in self.E and self.rewards[i] > best_reward:
                    best_reward = self.rewards[i]
                    best_resource = i
            if best_resource is not None:
                total_reward += best_reward
                # Sample the usage duration
                duration = np.random.choice(self.usage_duration_distributions[best_resource])
                resource_availability[best_resource] = t + duration
        return total_reward

    def clairvoyant_opt(self, arrival_sequence):
        total_reward = 0
        resource_availability = [0] * self.N
        for t in range(self.T):
            current_request = arrival_sequence[t]
            # Check all resources that can be matched with the current request
            for i in range(self.N):
                if (i, current_request) in self.E and resource_availability[i] <= t:
                    total_reward += self.rewards[i]
                    duration = np.random.choice(self.usage_duration_distributions[i])
                    resource_availability[i] = t + duration
                    break  # Assuming one match per request
        return total_reward


# Example usage
N = 5  # Number of resources
T = 10  # Number of requests
E = [(i, j) for i in range(N) for j in range(T) if random.random() > 0.5]  # Randomly generate edges
rewards = np.random.randint(1, 10, N)
usage_duration_distributions = [np.random.randint(1, 5, 10) for _ in range(N)]

model = OnlineMatchingModel(N, T, E, rewards, usage_duration_distributions)
greedy_reward = model.greedy_algorithm()
arrival_sequence = np.random.permutation(T)  # Random arrival sequence for OPT
opt_reward = model.clairvoyant_opt(arrival_sequence)

print(f"Greedy Reward: {greedy_reward}")
print(f"OPT Reward: {opt_reward}")


Greedy Reward: 49
OPT Reward: 49


#### Inventory Balancing Algorithm (with unreusable resources)

In [2]:
class OnlineMatchingModel:
    def __init__(self, N, T, E, rewards):
        self.N = N  # Number of offline vertices (resources)
        self.T = T  # Number of online vertices (requests)
        self.E = E  # Edges in the bipartite graph
        self.rewards = rewards  # Reward for each resource
        
        self.inventory = {}
        for n in range(N):
            self.inventory[n] = 1 # Each resource is initially set to available

        
    def g(self, x):
        return np.exp(-x)
    # Used to adjust the selected weight or priority of each resource
    # When the ratio of the resource's remaining inventory x is high, the value of e^(-x) is smaller, 
    # which means that the priority of resources being selected is correspondingly higher, 
    # because we are more inclined to use resources with more remaining inventory
    
    def allocate_resource(self, t):
        scores = {}
        for n in self.E[t]:
            if self.inventory[n] > 0:  # Allocation is considered only when resources are available
                score = (1 - self.g(self.inventory[n])) * self.rewards[n]
                scores[n] = score

        if scores:
            selected_resource = max(scores, key=scores.get)
            self.inventory[selected_resource] = 0  # After allocation, the resource is marked as unavailable
            return selected_resource
        return None  # If no resources are available, return None
    
# sample
N = 5
T = 10
E = {}
for t in range(T):
    E[t] = range(N)

rewards = np.random.rand(N)

model = OnlineMatchingModel(N, T, E, rewards)

allocations = []
for t in range(T):
    allocation = model.allocate_resource(t)
    allocations.append(allocation)

print(allocations)

[0, 3, 1, 4, 2, None, None, None, None, None]


#### Inventory Balancing Algorithm (reusable)

In [3]:
import numpy as np

class OnlineMatchingModel:
    def __init__(self, N, T, E, rewards, usage_duration_distributions):
        self.N = N  # Number of offline vertices (resources)
        self.T = T  # Number of online vertices (requests)
        self.E = E  # Edges in the bipartite graph
        self.rewards = rewards  # Reward for each resource
        self.usage_duration_distributions = usage_duration_distributions  # Usage duration distributions

        self.inventory = {}
        for n in range(N):
            self.inventory[n] = 1  # Each resource is initially set to available
        
        self.return_times = {}
        for n in range(N):
            self.return_times[n] = []  # Initializes the return time list for each resource
    
    def g(self, x):
        return np.exp(-x)
    # Used to adjust the selected weight or priority of each resource
    # When the ratio of the resource's remaining inventory x is high, the value of e^(-x) is smaller, 
    # which means that the priority of resources being selected is correspondingly higher, 
    # because we are more inclined to use resources with more remaining inventory

    
    def update_inventory(self, current_time):
        for n in range(self.N):
            updated_return_times = []
            for return_time in self.return_times[n]:
                if return_time > current_time:
                    updated_return_times.append(return_time)
            self.return_times[n] = updated_return_times
            
            if not updated_return_times:  # If there is no resource waiting to be returned, it is set to available
                self.inventory[n] = 1
            else:
                self.inventory[n] = 0
    
    def allocate_resource(self, t):
        self.update_inventory(t)
        
        scores = {}
        for n in self.E[t]:
            score = (1 - self.g(self.inventory[n])) * self.rewards[n]
            scores[n] = score
        
        selected_resource = max(scores, key=scores.get)
        
        duration = np.random.choice(self.usage_duration_distributions[selected_resource])
        self.return_times[selected_resource].append(t + duration)
        
        self.inventory[selected_resource] = 0
        
        return selected_resource

# sample
N = 5
T = 10
E = {}
for t in range(T):
    E[t] = range(N)

rewards = np.random.rand(N)

usage_duration_distributions = {}
for n in range(N):
    usage_duration_distributions[n] = [1, 2, 3]

model = OnlineMatchingModel(N, T, E, rewards, usage_duration_distributions)

allocations = []
for t in range(T):
    allocation = model.allocate_resource(t)
    allocations.append(allocation)

print(allocations)

[4, 3, 3, 4, 4, 3, 4, 4, 4, 3]


#### Rank Based Allocation Algorithm

In [4]:
import numpy as np

class OnlineMatchingModel:
    def __init__(self, N, T, E, rewards, usage_duration_distributions):
        self.N = N  # Number of offline vertices (resources)
        self.T = T  # Number of online vertices (requests)
        self.E = E  # Edges in the bipartite graph
        self.rewards = rewards  # Reward for each resource
        self.usage_duration_distributions = usage_duration_distributions  # Usage duration distributions
        
        # Initialize the availability and ranking of each unit of the resources
        self.availability = {n: [True] * self.usage_duration_distributions[n][0] for n in range(self.N)}
        # The ranking is simplified as a list of boolean values indicating availability

    def g(self, x):
        # Decreasing function g(x) = e^(-x)
        return np.exp(-x)
    
    def allocate_resource(self, t):
        scores = {}
        for i in self.E[t]:
            if any(self.availability[i]):  # Check if there's any unit available for resource i
                highest_available_unit = max([k for k, available in enumerate(self.availability[i]) if available])
                scores[i] = self.rewards[i] * (1 - self.g((highest_available_unit + 1) / len(self.availability[i])))

        if scores:
            selected_resource = max(scores, key=scores.get)
            # Mark the highest ranked available unit of the selected resource as unavailable
            highest_available_unit = max([k for k, available in enumerate(self.availability[selected_resource]) if available])
            self.availability[selected_resource][highest_available_unit] = False
            # Set a timer for when the unit will become available again, based on the usage duration distribution
            return selected_resource, self.usage_duration_distributions[selected_resource][0]  # Simplified: always use the first duration
        else:
            return None, None  # No resource is available

# Sample usage
N = 5
T = 10
E = {t: range(N) for t in range(T)}
rewards = np.random.rand(N)
usage_duration_distributions = {n: [np.random.choice([1, 2, 3])] for n in range(N)}

model = OnlineMatchingModel(N, T, E, rewards, usage_duration_distributions)

for t in range(T):
    resource, duration = model.allocate_resource(t)
    if resource is not None:
        print(f"Time {t}: Resource {resource} allocated for {duration} time units")
    else:
        print(f"Time {t}: No resource allocated")

Time 0: Resource 1 allocated for 3 time units
Time 1: Resource 2 allocated for 3 time units
Time 2: Resource 1 allocated for 3 time units
Time 3: Resource 2 allocated for 3 time units
Time 4: Resource 0 allocated for 1 time units
Time 5: Resource 1 allocated for 3 time units
Time 6: Resource 4 allocated for 3 time units
Time 7: Resource 2 allocated for 3 time units
Time 8: Resource 3 allocated for 1 time units
Time 9: Resource 4 allocated for 3 time units
